In [331]:
import swat
import os
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import lightgbm as lgb
import time
import gc
from sklearn.model_selection import train_test_split
from lightgbm import early_stopping
try:
    import cPickle as pickle
except BaseException:
    import pickle


import sasctl.pzmm as pzmm
from sasctl import Session
from sasctl.services import model_repository as modelRepo
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split
from pathlib import Path
from sasctl.services import model_repository as mr

In [332]:
os.environ['CAS_CLIENT_SSL_CA_LIST'] = "C:/Users/turkak/SASVIYACERT/my_ca_certificate.pem"
hostname = 'https://server.demo.sas.com:443/cas-shared-default-http/'
username = 'sasdemo'
password = 'Orion123'
conn = swat.CAS('https://server.demo.sas.com:443/cas-shared-default-http/', username='sasdemo',password='Orion123')

In [333]:
conn.serverStatus()

NOTE: Grid node action status report: 1 nodes, 9 total actions executed.


[About]

 {'CAS': 'Cloud Analytic Services',
  'CASCacheLocation': 'CAS Disk Cache',
  'CASHostAccountRequired': 'OPTIONAL',
  'Copyright': 'Copyright © 2014-2022 SAS Institute Inc. All Rights Reserved.',
  'ServerTime': '2022-07-31T05:43:02Z',
  'System': {'Hostname': 'controller.sas-cas-server-default.sas-viya.svc.cluster.local',
   'Linux Distribution': 'Red Hat Enterprise Linux release 8.6 (Ootpa)',
   'Model Number': 'x86_64',
   'OS Family': 'LIN X64',
   'OS Name': 'Linux',
   'OS Release': '3.10.0-1160.6.1.el7.x86_64',
   'OS Version': '#1 SMP Tue Nov 17 13:59:11 UTC 2020'},
  'Transferred': 'NO',
  'Version': '4.00',
  'VersionLong': 'V.04.00M0P07182022',
  'Viya Release': '20220721.1658437616927',
  'Viya Version': 'Stable 2022.1.3',
  'license': {'expires': '22Oct2022:00:00:00',
   'gracePeriod': 45,
   'site': 'VIYA 4 W ESP, ACCESS, VA, VDMML, AIOT, AND PQA',
   'siteNum': 70180938,
   'warningPeriod': 49}}

[nodestatus]

 Node Status
 
                                                 name        role  uptime  running  stalled
 0  controller.sas-cas-server-default.sas-viya.svc...  controller   7.419        0        0

[server]

 Server Status
 
    nodes  actions
 0      1        9

+ Elapsed: 0.00083s, sys: 0.000803s, mem: 0.322mb

In [335]:
if not os.path.exists('Models'):
    os.makedirs('Models')

In [383]:
tbl = conn.read_csv("./datasets/hmeq.csv",casout=dict(name='hmeq'))


NOTE: Cloud Analytic Services made the uploaded file available as table HMEQ in caslib CASUSER(sasdemo).
NOTE: The table HMEQ has been created in caslib CASUSER(sasdemo) from binary data uploaded to Cloud Analytic Services.


In [385]:
import time
start_time = time.time()
df = tbl.to_frame()
print("--- %s seconds ---" % (time.time() - start_time))

--- 5.170612573623657 seconds ---


In [339]:
def batch(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]
        

 

In [340]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
#pd_df= pd.DataFrame(tbl.fetch(**fetch_opts)['Fetch'])
#pd_df= reduce_mem_usage(pd_df)

In [390]:
pd_df= pd.DataFrame(reduce_mem_usage(tbl.to_frame()))

Mem. usage decreased to  0.30 Mb (53.6% reduction)


In [392]:
#conn.columninfo(table=dict(name='hmeq'))

Mem. usage decreased to  0.04 Mb (53.6% reduction)
Mem. usage decreased to  0.04 Mb (53.6% reduction)
Mem. usage decreased to  0.04 Mb (53.6% reduction)
Mem. usage decreased to  0.04 Mb (53.6% reduction)
Mem. usage decreased to  0.04 Mb (55.4% reduction)
Mem. usage decreased to  0.04 Mb (55.4% reduction)
Mem. usage decreased to  0.04 Mb (55.4% reduction)


In [ ]:
#model= lgb.train(params=lgb_params,train_set=dtrain_set,num_boost_round=1000,valid_sets=dtest_set,early_stopping_rounds=20,verbose_eval=10)

In [411]:
def incremental_learning(data):
    start = time.time()
    params = {
 'boosting_type': 'gbdt',
 'objective': 'binary',
 'metric': 'binary_logloss',
 'num_leaves': 75,
 'learning_rate': 0.05,
 'feature_fraction': 0.75,
 'bagging_fraction': 0.75,
 'bagging_freq': 0,
 'min_data_per_group': 10
    }

    X_tr, X_te, y_tr, y_te = train_test_split(
        data, 
        data['BAD'], 
        test_size=0.2)
    del data
    files = os.listdir('./Models/')

    
    
    dtrain = lgb.Dataset(X_tr, y_tr, free_raw_data=False)
    dtest_set = lgb.Dataset(X_te, y_te, free_raw_data=False)

    if "model.pkl" not in files:
        print("No Model yet...")
        model = lgb.train(params=params,train_set=dtrain,callbacks=[early_stopping(5)],valid_sets=dtest_set,verbose_eval=10)
        with open('./Models/model.pkl', 'wb') as fout:
            pickle.dump(model, fout)
    
    with open('./Models/model.pkl', 'rb') as fin:
        pkl_bst = pickle.load(fin)
    model = lgb.train(params, 
                       init_model = pkl_bst,
                       train_set = dtrain,
                       valid_sets=dtest_set,
                       keep_training_booster = True,
                      
                     callbacks=[early_stopping(5)])
    #f1 = f1_score(y_te, model.predict(X_te))
    #print("f1_score:", f1)
    
    return model
    
    

['REASON', 'JOB']

In [412]:
batch_rows = 1000
maxn= 6000
dflist = list()
for x in batch(range(0, maxn), batch_rows):
    fetch_opts = dict(maxrows=maxn,from_=x[0] + 1 ,to=x[0] + batch_rows )
    if x[0] ==0:
        fetch_opts = dict(maxrows=maxn,from_=x[0]  ,to=x[0] + batch_rows )
    df = pd.DataFrame(reduce_mem_usage(tbl.to_frame(**fetch_opts)))
    dtypes = df.dtypes
    nominals = dtypes[dtypes=='object'].keys().tolist()
    if len(nominals) >0:
        for col in nominals:
            df[col] = df[col].astype('category')
    model = incremental_learning(df)
    with open('./Models/model.pkl', 'wb') as fout:
        pickle.dump(model, fout)
    dflist.append(model)    
    del model
    gc.collect()
    #df = pd.DataFrame(tbl.fetch(**fetch_opts)['Fetch'])
    

Mem. usage decreased to  0.05 Mb (53.6% reduction)
No Model yet...
[LightGBM] [Info] Number of positive: 241, number of negative: 559
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000337 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1096
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.301250 -> initscore=-0.841353
[LightGBM] [Info] Start training from score -0.841353
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG

C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Number of positive: 241, number of negative: 559
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000225 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1096
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 13
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain,

C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Mem. usage decreased to  0.05 Mb (53.6% reduction)
[LightGBM] [Info] Number of positive: 162, number of negative: 638
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000297 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1136
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 13
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No 

...

C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Mem. usage decreased to  0.05 Mb (55.4% reduction)
[LightGBM] [Info] Number of positive: 115, number of negative: 685
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000381 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1155
[LightGBM] [Info] Number of data points in the train set: 800, number of used features: 13
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirem

C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


Mem. usage decreased to  0.05 Mb (53.6% reduction)
[LightGBM] [Info] Number of positive: 137, number of negative: 631
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000140 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1308
[LightGBM] [Info] Number of data points in the train set: 768, number of used features: 13
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warnin

C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[('valid_0', 'binary_logloss', 7.979287729799823e-05, False)]

In [319]:


model = incremental_learning(pd_df)

No Model yet...
[LightGBM] [Info] Number of positive: 477, number of negative: 1523
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000387 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1295
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.238500 -> initscore=-1.160921
[LightGBM] [Info] Start training from score -1.160921
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\lightgbm\engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\lightgbm\basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\lightgbm\basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Info] Number of positive: 477, number of negative: 1523
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000204 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1295
[LightGBM] [Info] Number of data points in the train set: 2000, number of used features: 13
Training until validation scores don't improve for 5 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [320]:
mm_python('OS_PROJECT','LGB_Model',model,'BAD',X_tr,y_tr,'Core Lightgbm Model','Lightgbm')

C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\sasctl\pzmm\writeJSONFiles.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outputJSON = outputJSON.append([outputRow], ignore_index=True)
C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\sasctl\pzmm\writeJSONFiles.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outputJSON = outputJSON.append([outputRow], ignore_index=True)
C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\sasctl\pzmm\writeJSONFiles.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outputJSON = outputJSON.append([outputRow], ignore_index=True)
C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\sasctl\pzmm\writeJSONFiles.py:104: FutureWarning: The frame.a

inputVar.json was successfully written and saved to C:\Users\turkak\Projects\akbank_poc\Outputs\LGB_Model\inputVar.json
outputVar.json was successfully written and saved to C:\Users\turkak\Projects\akbank_poc\Outputs\LGB_Model\outputVar.json
fileMetaData.json was successfully written and saved to C:\Users\turkak\Projects\akbank_poc\Outputs\LGB_Model\fileMetaData.json
ModelProperties.json was successfully written and saved to C:\Users\turkak\Projects\akbank_poc\Outputs\LGB_Model\ModelProperties.json
dmcas_fitstat.json was successfully written and saved to C:\Users\turkak\Projects\akbank_poc\Outputs\LGB_Model\dmcas_fitstat.json
NOTE: Added action set 'percentile'.
NOTE: Cloud Analytic Services made the uploaded file available as table SCOREDVALUES in caslib CASUSER(sasdemo).
NOTE: The table SCOREDVALUES has been created in caslib CASUSER(sasdemo) from binary data uploaded to Cloud Analytic Services.
dmcas_roc.json was successfully written and saved to C:\Users\turkak\Projects\akbank_poc\

Traceback (most recent call last):
  File "C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\turkak\AppData\Local\Temp\ipykernel_21044\2765167922.py", line 1, in <cell line: 1>
    mm_python('OS_PROJECT','LGB_Model',model,'BAD',X_tr,y_tr,'Core Lightgbm Model','Lightgbm')
  File "C:\Users\turkak\AppData\Local\Temp\ipykernel_21044\3578387189.py", line 87, in mm_python
    modelRepo.import_model_from_zip(model_name, project_name, zipIOFile,version='latest')
  File "C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\sasctl\_services\model_repository.py", line 527, in import_model_from_zip
    r = cls.post(
  File "C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\sasctl\_services\service.py", line 113, in post
    return cls.request("post", *args, **kwargs)
  File "C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\sasctl\_se

In [315]:
mr.create_project(project='OS_PROJECT',repository='DMRepository')

<class 'sasctl.core.RestObj'>(headers={'Date': 'Sat, 30 Jul 2022 14:18:20 GMT', 'Content-Type': 'application/vnd.sas.models.project+json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Origin, Access-Control-Request-Method, Access-Control-Request-Headers', 'SAS-Service-Response-Flag': 'true', 'Content-Security-Policy': "default-src 'self'; object-src 'none';", 'Last-Modified': 'Sat, 30 Jul 2022 14:18:19 GMT', 'ETag': '"l67zaxl7"', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'X-Frame-Options': 'SAMEORIGIN'}, data={'creationTimeStamp': '2022-07-30T14:18:19.610Z', 'modifiedTimeStamp': '2022-07-30T14:18:19.915Z', 'createdBy': 'sasdemo', 'modifiedBy': 'sasdemo', 'links': [{'method': 'GET', 'rel': 'up', 'href': '/modelRepository/projects', 'uri': '/modelRepository/proje

In [262]:
hostname = 'server.demo.sas.com'

In [322]:
def mm_python(project_name, model_name, model, target, input_df, actual_df, desc, model_type):      
    ##################################################
    #input_df[cat_cols] = input_df[cat_cols].astype('str')
    # CREATE OUTPUT INFORMATION  

    # Output folder name
    output_folder = 'Outputs'

    # Create Folder
    output_path = Path.cwd() / output_folder / model_name
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    ##################################################
    # CONNECT TO SAS VIYA
    host_session = 'https://' + hostname + '/'
    
    # Connect using a session 
   # sess=Session(host_session, username, password, verify_ssl=False, protocol="http")
    sess=Session(host_session, username, password)
    ##################################################
    # COMPILE METADATA

    # Create JSON Files Object
    JSONFiles = pzmm.JSONFiles()

    # Write Input Variable JSON 
    JSONFiles.writeVarJSON(input_df, isInput=True, jPath=output_path)

    # Mock up Output Variables
    yCategory = actual_df.astype('str') # Ensuring dataframe length matches
    output_df = pd.DataFrame(columns=['EM_EVENTPROBABILITY', 'EM_CLASSIFICATION']) # Ensuring column names are what SAS expects
    output_df['EM_CLASSIFICATION'] = yCategory # Ensuring data type is nominal 
    output_df['EM_EVENTPROBABILITY'] = 0.5 # Ensuring data type is decimal  

    # Write Output Variable JSON
    JSONFiles.writeVarJSON(output_df, isInput=False, jPath=output_path)

    # Write File Metadata JSON
    JSONFiles.writeFileMetadataJSON(model_name, jPath=output_path)

    # Write Model Properties JSON
    JSONFiles.writeModelPropertiesJSON(modelName=model_name, 
        modelDesc=desc,
        targetVariable=target,
        modelType=model_type,
        modelPredictors=input_df.columns.array,
        targetEvent='1',
        numTargetCategories=2,
        eventProbVar='EM_EVENTPROBABILITY',
        jPath=output_path,
        modeler=username)

    # Get predictions
    trainProba = model.predict(input_df)

    # Creating Assessment Data 
    trainData = pd.concat([pd.DataFrame(actual_df).reset_index(drop=True), pd.Series(data=trainProba)], axis=1)

    # Write Fit Statistics JSON
    JSONFiles.calculateFitStat(trainData=trainData, jPath=output_path)

    # Write ROC amd Lift Statistics JSON
    JSONFiles.generateROCLiftStat(target, 1, conn, trainData=trainData, jPath=output_path)

    ##################################################
    # PICKLE MODEL 
    pzmm.PickleModel.pickleTrainedModel(_, model, model_name, output_path)

    ##################################################
     #WRITE SCORE CODE 

    # Generate Score Code Object
    ScoreCode = pzmm.ScoreCode()

    # Write Score Code
    ScoreCode.writeScoreCode(input_df, actual_df, model_name, '{}.predict({})', model_name + '.pickle', pyPath=output_path)

    ##################################################
    # ZIP FOLDER
    
    zipIOFile = pzmm.ZipModel.zipFiles(fileDir=output_path, modelPrefix=model_name)

    ##################################################
    # PUSH ZIP FOLDER INTO MODEL MANAGER
    with sess: 
        modelRepo.import_model_from_zip(model_name, project_name, zipIOFile,version='new')

In [300]:
 trainData = pd.concat([pd.DataFrame(y_tr), pd.DataFrame(trainProba)], axis=1)

In [256]:
X_tr, X_te, y_tr, y_te = train_test_split(
        pd_df, 
        pd_df['BAD'], 
        test_size=0.2)


In [323]:
mm_python('OS_PROJECT','LGB_Model',model,'BAD',X_tr,y_tr,'Core Lightgbm Model','Lightgbm')

C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\sasctl\pzmm\writeJSONFiles.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outputJSON = outputJSON.append([outputRow], ignore_index=True)
C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\sasctl\pzmm\writeJSONFiles.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outputJSON = outputJSON.append([outputRow], ignore_index=True)
C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\sasctl\pzmm\writeJSONFiles.py:104: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  outputJSON = outputJSON.append([outputRow], ignore_index=True)
C:\Users\turkak\Anaconda3\envs\akbank_poc\lib\site-packages\sasctl\pzmm\writeJSONFiles.py:104: FutureWarning: The frame.a

inputVar.json was successfully written and saved to C:\Users\turkak\Projects\akbank_poc\Outputs\LGB_Model\inputVar.json
outputVar.json was successfully written and saved to C:\Users\turkak\Projects\akbank_poc\Outputs\LGB_Model\outputVar.json
fileMetaData.json was successfully written and saved to C:\Users\turkak\Projects\akbank_poc\Outputs\LGB_Model\fileMetaData.json
ModelProperties.json was successfully written and saved to C:\Users\turkak\Projects\akbank_poc\Outputs\LGB_Model\ModelProperties.json
dmcas_fitstat.json was successfully written and saved to C:\Users\turkak\Projects\akbank_poc\Outputs\LGB_Model\dmcas_fitstat.json
NOTE: Added action set 'percentile'.
NOTE: Cloud Analytic Services made the uploaded file available as table SCOREDVALUES in caslib CASUSER(sasdemo).
NOTE: The table SCOREDVALUES has been created in caslib CASUSER(sasdemo) from binary data uploaded to Cloud Analytic Services.
dmcas_roc.json was successfully written and saved to C:\Users\turkak\Projects\akbank_poc\